## 랜덤 포레스트 - 분류

- 결정 트리(나무)로 랜덤하게 랜덤 포레스트(숲)를 만듦
- 각 결정 트리의 예측을 사용해 최종 예측을 만듦

- 부트스트랩 샘플(중복허용 샘플링)
  - 랜덤하게 중복 가능하게 추출
  - 기본적으로 훈련 세트와 부트스트랩 샘플의 크기를 같게 만듦  

### 1. 데이터 세트 나누기

In [1]:
# 라이브러리 불러오기
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
# 데이터 불러오기
wine = pd.read_csv('https://bit.ly/wine-date')
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [3]:
# 속성 데이터 
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
data

array([[ 9.4 ,  1.9 ,  3.51],
       [ 9.8 ,  2.6 ,  3.2 ],
       [ 9.8 ,  2.3 ,  3.26],
       ...,
       [ 9.4 ,  1.2 ,  2.99],
       [12.8 ,  1.1 ,  3.34],
       [11.8 ,  0.8 ,  3.26]])

In [4]:
# 타깃 데이터
target = wine['class'].to_numpy()
target

array([0., 0., 0., ..., 1., 1., 1.])

In [5]:
# 세트 나누기
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size = 0.2, random_state = 42)

### 2. 교차검증 수행(RandomForestClassifier)

- 기본적으로 전체 특성 개수의 제곱근만큼의 특성을 선택

In [6]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs = -1, random_state = 42)
rf

RandomForestClassifier(n_jobs=-1, random_state=42)

In [7]:
scores = cross_validate(rf, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


### 3. 특성 중요도

In [8]:
rf.fit(train_input, train_target) # 훈련
print(rf.feature_importances_) # 특성중요도 출력

[0.23167441 0.50039841 0.26792718]


### 4. OOB 점수
**Out Of Bag 샘플 `(oob_score = True)`**
- 부트스트랩 샘플에 포함되지 않고 남는 샘플
- 부트스트랩 샘플로 훈련한 <span style = 'color:orange'>결정 트리를 평가</span>할 수 있음(검증 세트의 역할)

In [9]:
rf = RandomForestClassifier(oob_score = True, n_jobs = -1, random_state = 42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


In [10]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs = -1, random_state = 42)
scores = cross_validate(et, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [11]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


## 엑스트라 트리

In [12]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs = -1, random_state = 42)
scores = cross_validate(et, train_input, train_target,
                        return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [13]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


## 그레이디언트 부스팅

In [14]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state = 42)
scores = cross_validate(gb, train_input, train_target,
                        return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [15]:
gb = GradientBoostingClassifier(n_estimators = 500, learning_rate = 0.2,
                                random_state = 42)
scores = cross_validate(gb, train_input, train_target,
                        return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [16]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68011572 0.16116151]


## 히스토그램 기반 그레이디언트 부스팅

In [17]:
# from sklearn.experimental import enable_hist_gradient_boosting -> 불러오지 않아도 sklearn.ensemble에서 직접 가져올 수 있음
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state = 42)
scores = cross_validate(hgb, train_input, train_target,
                        return_train_score = True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [18]:
hgb.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [19]:
hgb.score(test_input, test_target)

0.8723076923076923

In [20]:
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method = 'hist', random_state = 42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9558403027491312 0.8782000074035686


## LightGBM

In [24]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state = 42)
scores = cross_validate(lgb, train_input, train_target, 
                        return_train_score = True, n_jobs = -1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884
